# <a id='toc1_'></a>[Generalized Meta-Framework for anomaly detection in time series data](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Generalized Meta-Framework for anomaly detection in time series data](#toc1_)    
  - [General functions](#toc1_1_)    
    - [package imports](#toc1_1_1_)    
    - [Dataset loading](#toc1_1_2_)    
    - [preprocessing](#toc1_1_3_)    
    - [Visualization](#toc1_1_4_)    
  - [TS Feature Extraction](#toc1_2_)    
  - [Model Training](#toc1_3_)    
    - [Splitting of dataset](#toc1_3_1_)    
    - [Exponential Smoothing](#toc1_3_2_)    
    - [ARIMA](#toc1_3_3_)    
    - [XGBoost](#toc1_3_4_)    
    - [Visualization of model prediction](#toc1_3_5_)    
  - [Stacking Approach for Predictions](#toc1_4_)    
    - [Random forest with random search hyperparameter tuning](#toc1_4_1_)    
    - [Visualization for stacking](#toc1_4_2_)    
    - [Running of stacking Approach](#toc1_4_3_)    
  - [Some temporary preparations for running ensemble size moldel](#toc1_5_)    
  - [Ensemble size finding model running](#toc1_6_)    
  - [ Running the final function and producing results](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[General functions](#toc0_)

### <a id='toc1_1_1_'></a>[package imports](#toc0_)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools
import random
import requests
import os
import json
import time
import psutil

from itertools import product
from datetime import datetime
from sklearn.impute import KNNImputer
from tsfeatures import tsfeatures
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools
import random
import requests
import os


from tsfeatures import tsfeatures
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

In [3]:
local_base_dir = 'data'
data = {}

def addFolderAndReadAll(d_name):
    data[d_name] = {}
    folder_path = os.path.join(local_base_dir, d_name)
    
    if not os.path.exists(folder_path):
        print(f"Directory {folder_path} does not exist.")
        return 0

    csv_files = [f_name for f_name in os.listdir(folder_path) if f_name.endswith('.csv')]
    csvs_num = 0
    for f_name in csv_files:
        file_path = os.path.join(folder_path, f_name)
        df = pd.read_csv(file_path)
        data[d_name][f_name] = df
        csvs_num += 1
    return csvs_num

directories = ["artificialWithAnomaly", "artificialNoAnomaly", "realAdExchange", "realAWSCloudwatch", "realKnownCause", "realTraffic", "realTweets"]
csvs_num = sum([addFolderAndReadAll(d_name) for d_name in directories])

print(f"Total CSV files processed: {csvs_num}")

Total CSV files processed: 58


In [4]:
# Function to get a random start date from the DataFrame index
def get_random_start_date(index):
    return np.random.choice(index)

# Main function to repeat the process until non-None frequency is obtained
def find_non_none_frequency(df, offset=9):
    while True:
        # Get a random start date from the DataFrame index
        start_date = pd.to_datetime(get_random_start_date(df.index))

        # Find the index of the end date by moving 9 steps through the indices
        end_date_index = df.index.get_loc(start_date) + offset

        # Check if the end date index is within the range of the DataFrame index
        if end_date_index < len(df.index):
            # Calculate the end date using the index
            end_date = df.index[end_date_index]

            # Infer frequency within the specified date range
            subset_df = df.loc[start_date:end_date]
            freq = pd.infer_freq(subset_df.index)

            if freq is not None:
                print("Inferred frequency within range", start_date, "-", end_date, ":", freq)
                return freq  # Exit the loop and return the inferred frequency

In [5]:
def max_consecutive_missing_dates(inferred_freq, missing_dates):
    # Function to check if two dates are consecutive based on the inferred frequency
    def are_consecutive(date1, date2, freq):
        # Calculate the difference between dates based on the inferred frequency
        diff = date2 - date1
        # Check if the difference matches the frequency
        if freq == 'D':
            return diff.days == 1
        elif freq.endswith('H')| freq.endswith('h'):
             # If the frequency ends with 'H', check if it represents hourly intervals
            if freq[:-1]:  # Check if there is a multiplier
                  interval = int(freq[:-1])
                  return diff.total_seconds() == interval * 3600
            else:
                   # If no multiplier is provided, it's assumed to be one hour
                   return diff.total_seconds() == 3600
        elif freq.endswith('T') | freq.endswith('min') :
            if freq.endswith('T'):
                # Extract the interval from the frequency string
                interval = int(freq[:-1])
                return diff.seconds // 60 == interval
            else:
                interval = int(freq[:-3])
                return diff.seconds // 60 == interval
        else:
            raise ValueError("Unsupported frequency: {}".format(freq))

    # Initialize variables to track maximum length and current length
    max_consecutive_missing = 0
    current_consecutive_missing = 0

    # Iterate over the missing dates
    for i in range(1, len(missing_dates)):
        # Check if the current date is consecutive with the previous date
        if are_consecutive(missing_dates[i - 1], missing_dates[i], inferred_freq):
            # Increment current consecutive missing count
            current_consecutive_missing += 1
        else:
            # Update maximum consecutive missing count if needed
            max_consecutive_missing = max(max_consecutive_missing, current_consecutive_missing)
            # Reset current consecutive missing count
            current_consecutive_missing = 0

    # Update max_consecutive_missing if current_consecutive_missing is still greater
    max_consecutive_missing = max(max_consecutive_missing, current_consecutive_missing)

    return max_consecutive_missing

In [6]:
def preprocess(df, f_name):
    # Convert 'timestamp' column to datetime format and rename it to 'ds'
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Removing the duplicate rows
    df = df[~df.duplicated(keep='first')]

    duplicated_dates_length = len(df[df['timestamp'].duplicated(keep=False)])

    if  duplicated_dates_length > 0:
      print("Number of Duplicated Dates in "+ f_name + ": "+ str(duplicated_dates_length))
      # To make the mean as the value for the numerical columns if there are different values for a particular date
      df = df.groupby('timestamp').mean()
      # Reset index to bring 'timestamp' column back
      df.reset_index(inplace=True)

    df.set_index(['timestamp'], inplace=True)
    df.sort_index()

    # Create a date range with hourly frequency covering the entire time range
    start_date = df.index.min()
    end_date = df.index.max()

    #inferred_freq = pd.infer_freq(df.index)
    inferred_freq = find_non_none_frequency(df)

    if inferred_freq is None:
      inferred_freq = default_freq # setting the default frequency
      print("Cannot infer the frequency of the timestamp of the dataset "+ f_name+ " .Therefore the default frequency of " + default_freq+ " will be used")

    expected_date_range = pd.date_range(start=start_date, end=end_date, freq=inferred_freq)

    # Find the missing date entries
    missing_dates = expected_date_range[~expected_date_range.isin(df.index)]
    # Print or work with the list of missing dates
    print("Number of Missing Dates in "+ f_name + ": "+ str(len(missing_dates))+"\n")

    if len(missing_dates) > 0:
      df = df.asfreq(inferred_freq)
      df.sort_index()

      # Call the function with inferred_freq and missing_dates parameters
      max_consecutive = max_consecutive_missing_dates(inferred_freq, missing_dates)
      print("Maximum length of consecutive missing dates:", max_consecutive)
      if max_consecutive > 3:
        print("It is better to use other imputation method rather than linear interpolation")

      df['value'] = df['value'].interpolate(method='linear')

    return df

In [7]:
url = 'https://raw.githubusercontent.com/numenta/NAB/master/labels/combined_labels.json'

response = requests.get(url)

if response.status_code == 200:
    labels = json.loads(response.text)
else:
    print("Failed to retrieve data from the URL:", response.status_code)

In [8]:
# List of directories
dirs = ['realAdExchange', 'realAWSCloudwatch', 'realKnownCause', 'realTweets', 'artificialWithAnomaly', 'artificialNoAnomaly', 'realTraffic']
#dirs = ['realAdExchange']
#dirs = ['artificialNoAnomaly']

# Loop through each directory
for dir in dirs:
    for f_name in data[dir]:
        print("")
        print(f"Iterating over file: {dir} / {f_name}")
        df = preprocess(data[dir][f_name], f_name)
        labels_of_one_file = labels[dir+'/'+f_name]
        df['is_anomaly'] = 0
        for anomalous_timestamp in labels_of_one_file:
            anomalous_timestamp = pd.to_datetime(anomalous_timestamp)
            try:
                df.at[anomalous_timestamp, 'is_anomaly'] = 1  # Set is_anomaly to 1 at the index location
            except KeyError:
                print(f"Anomalous timestamp {anomalous_timestamp} not found in data[{dir}][{f_name}].")
                pass
        data[dir][f_name] = df  # Assign the modified DataFrame back to the data dictionary


Iterating over file: realAdExchange / exchange-2_cpm_results.csv
Number of Duplicated Dates in exchange-2_cpm_results.csv: 2
Inferred frequency within range 2011-08-30 13:00:01 - 2011-08-30 22:00:01 : h
Number of Missing Dates in exchange-2_cpm_results.csv: 25

Maximum length of consecutive missing dates: 19
It is better to use other imputation method rather than linear interpolation

Iterating over file: realAdExchange / exchange-3_cpc_results.csv
Inferred frequency within range 2011-08-31 07:15:01 - 2011-08-31 16:15:01 : h
Number of Missing Dates in exchange-3_cpc_results.csv: 109

Maximum length of consecutive missing dates: 14
It is better to use other imputation method rather than linear interpolation

Iterating over file: realAdExchange / exchange-4_cpm_results.csv
Inferred frequency within range 2011-07-07 14:15:01 - 2011-07-07 23:15:01 : h
Number of Missing Dates in exchange-4_cpm_results.csv: 4

Maximum length of consecutive missing dates: 0

Iterating over file: realAdExchan

In [9]:
# Specify the directory and file name
dir = 'realAWSCloudwatch'
f_name = 'ec2_cpu_utilization_ac20cd.csv'


# Retrieve the DataFrame
df = data[dir][f_name]

In [10]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-04-15')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df


In [11]:
f_name = 'ec2_cpu_utilization_5f5533.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

In [12]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-02-25')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

In [13]:
f_name = 'grok_asg_anomaly.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

In [14]:
cutoff_date = pd.Timestamp('2014-01-29')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

In [15]:
f_name = 'rds_cpu_utilization_cc0c53.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

In [16]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-02-25')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

In [17]:
def split_data(df, train_ratio=0.6):
    train_size = int(len(df) * train_ratio)
    train, val = df[:train_size], df[train_size:]
    return train, val

In [18]:
original_values_df = pd.DataFrame(columns=['dir', 'values'])

for dir in dirs:
    print(f"Iterating over directory: {dir}")

    # Iterate over each file in the current directory
    for file_name in data[dir]:
        df = data[dir][file_name]
        if dir == 'realTraffic':
            df = df.dropna()
        # Assuming df is defined somewhere in the loop
        train, val = split_data(df)

        # Create a new row with the directory and values as a list
        new_row = pd.DataFrame({'dir': [dir], 'file_name':[file_name], 'values': [val['value'].tolist()]})

        # Concatenate the new row to the original_values_df
        original_values_df = pd.concat([original_values_df, new_row], ignore_index=True)


original_values_df

Iterating over directory: realAdExchange
Iterating over directory: realAWSCloudwatch
Iterating over directory: realKnownCause
Iterating over directory: realTweets
Iterating over directory: artificialWithAnomaly
Iterating over directory: artificialNoAnomaly
Iterating over directory: realTraffic


,dir,values,file_name
0,realAdExchange,"[0.160230773158, 0.210830475115, 0.28622422434...",exchange-2_cpm_results.csv
1,realAdExchange,"[0.0970408163265, 0.0936600833484, 0.094246275...",exchange-3_cpc_results.csv
2,realAdExchange,"[0.428944811047, 0.469532759242, 0.45564022899...",exchange-4_cpm_results.csv
3,realAdExchange,"[0.0737386325362, 0.0840131819846, 0.079394993...",exchange-4_cpc_results.csv
4,realAdExchange,"[0.0848198198198, 0.0985912823752, 0.111722365...",exchange-2_cpc_results.csv
5,realAdExchange,"[0.56535245238, 0.545612490685, 0.521651179882...",exchange-3_cpm_results.csv
6,realAWSCloudwatch,"[0.102, 0.102, 81.468, 98.67, 60.688, 87.11200...",ec2_cpu_utilization_77c1ca.csv
7,realAWSCloudwatch,"[5.834, 7.066, 5.856, 6.228, 5.636, 6.058, 5.8...",rds_cpu_utilization_cc0c53.csv
8,realAWSCloudwatch,"[47.018, 43.004, 41.066, 44.554, 43.284, 42.12...",ec2_cpu_utilization_5f5533.csv
9,realAWSCloudwatch,"[219840.0, 209992.0, 215690.0, 182444.0, 21823...",ec2_network_in_257a54.csv


In [19]:
df = pd.read_csv('test/test.csv')
df.head() 

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,exponential_smoothing,xgboost,random_forest,gru,vae,lstm
0,realAdExchange/exchange-4_cpm_results.csv,0.390779,1647,-1839.697103,0.181736,0.006275,2.640000e-12,5.940000e-15,0.011183,1,...,0.263340,-0.671191,0.494804,0.000060,0.209124,0.200501,0.221373,0.172731,0.292100,0.170313
1,realAWSCloudwatch/ec2_cpu_utilization_77c1ca.csv,0.950880,4032,-654.999557,0.489215,0.999912,3.790000e-06,1.980000e-07,0.076766,1,...,0.098248,-0.371219,0.192937,0.116130,15.773876,22.753666,21.003871,5.856619,6.754569,5.778467
2,realAWSCloudwatch/ec2_cpu_utilization_ac20cd.csv,1.089914,4037,-8.202838,12.161875,0.435310,9.020000e-11,9.420000e-08,0.834827,1,...,0.279253,-0.680912,0.529230,0.320249,1.580107,2.292306,5.128382,1.621384,1.600771,1.606470
3,realAWSCloudwatch/ec2_cpu_utilization_fe7f93.csv,0.802616,4032,-992.787092,0.200876,0.999887,8.840000e-07,1.700000e-07,0.029572,1,...,0.188679,-0.326595,0.247232,0.084554,3.390697,6.589674,6.827590,1.686439,6.839780,1.574349
4,realKnownCause/ambient_temperature_system_fail...,1.085836,7888,-157.619180,15.832975,0.701509,2.860000e-05,1.133530e-04,0.698239,1,...,0.099559,-0.645464,0.443414,0.523375,4.476763,5.694003,6.423569,0.714506,1.296207,0.761606


In [20]:
predicted_df_final = pd.read_csv('test/predicted_results.csv')
predicted_df_final.head()

,dir,file_name,original_value,exponential_smoothing,xgboost,random_forest,lstm,gru,vae
0,realKnownCause,ambient_temperature_system_failure.csv,[73.41217429 72.83454005 73.5939262 ... 72.04...,"[73.77850842954233, 73.20902976953136, 72.0610...","[74.10153198242188, 73.49702453613281, 72.5597...","[74.56356811523438, 74.153076171875, 73.547012...","[0.5542659759521484, 0.5628800392150879, 0.570...","[0.5222489237785339, 0.5272617340087891, 0.540...","[73.41217429, 72.83454005, 73.5939262, 72.7640..."
1,realAWSCloudwatch,ec2_cpu_utilization_77c1ca.csv,[ 0.102 0.102 81.468 ... 0.102 0.1 0.102],"[10.067944417212566, 9.17267661760961, 17.1446...","[12.926828384399414, 12.926828384399414, 12.92...","[0.09017550945281982, 0.09017550945281982, 0.0...","[0.0, 0.0, 1.0730327367782593, 0.8917079567909...","[-0.002159492578357458, -0.0016223951242864132...","[0.10200000000000031, 0.10200000000000031, 81...."
2,realAWSCloudwatch,ec2_cpu_utilization_ac20cd.csv,[34.6 36.854 35.822 ... 53.307875 5...,"[33.21722971569003, 33.58044325157902, 35.1385...","[33.95431137084961, 33.86821746826172, 33.8682...","[34.877044677734375, 34.86305618286133, 34.863...","[0.5955383777618408, 0.5939427018165588, 0.593...","[0.5873817801475525, 0.5895547270774841, 0.586...","[34.6, 36.854, 35.822, 34.226, 39.906, 30.83, ..."
3,realAWSCloudwatch,ec2_cpu_utilization_fe7f93.csv,[2.332 2.022 2.278 ... 2.376 2.426 3.252],"[2.053437398786812, 2.450225009871555, 3.36526...","[28.57091522216797, 28.57091522216797, 28.5709...","[33.21626281738281, 33.21626281738281, 33.2162...","[0.004909306298941374, 0.005058541428297758, 0...","[0.0040578520856797695, 0.004336375277489424, ...","[2.332, 2.0219999999999994, 2.2780000000000005..."
4,realAdExchange,exchange-4_cpm_results.csv,[ 0.42894481 0.46953276 0.45564023 0.407852...,"[0.4391361259348808, 0.47813139304474145, 0.51...","[0.4909343123435974, 0.4909343123435974, 0.490...","[0.4492344260215759, 0.48435816168785095, 0.47...","[0.013979798182845116, 0.013344982638955116, 0...","[0.011760089546442032, 0.010348841547966003, 0...","[0.428944811047, 0.469532759242, 0.45564022899..."


### <a id='toc1_3_5_'></a>[Visualization of model prediction](#toc0_)

In [21]:
def plot_forecast_interactive(forecast_df, val, model_name):

    # Create a directory if it doesn't exist
    output_folder = os.path.join("visualization", model_name)
    os.makedirs(output_folder, exist_ok=True)

    # Plot forecast and real values
    forecast_trace = go.Scatter(x=forecast_df.index, y=forecast_df['Forecast'], mode='lines', name='Forecast')
    real_trace = go.Scatter(x=val.index, y=val['value'], mode='lines', name='Real')

    # Create the layout
    layout = go.Layout(title=f"{model_name}",xaxis=dict(title='Timestamp'), yaxis=dict(title='Value'))

    # Combine traces into a list
    data = [forecast_trace, real_trace]

    # Create the figure
    fig = go.Figure(data=data, layout=layout)

    # Show the interactive plot
    fig.show()

## <a id='toc1_4_'></a>[Stacking Approach for Predictions](#toc0_)

### <a id='toc1_4_1_'></a>[Random forest with random search hyperparameter tuning](#toc0_)

In [22]:
def stacked_model_predictions(val, base_preds):
     # Convert string inputs to lists of floats
     # Convert string inputs to lists of floats for each column in base_preds
    base_preds = [list(map(float, col.strip('[]').split(','))) for col in base_preds]
    
    # Convert lists to numpy arrays and transpose to get the correct shape
    base_preds = np.asarray(base_preds).T

    print("Type of base_preds:", type(base_preds))
    print("Shape of base_preds:", base_preds.shape)
    #val = list(map(float, val[0].strip('[]').split(',')))
    
    
    
    # Convert lists to numpy arrays
    val = np.asarray(val).reshape(-1)
    #base_preds = np.asarray(base_preds).reshape(-1, 1)
    # Check the type and shape after conversion
    print("Type of val:", type(val))
    print("Shape of val:", val.shape)
    

    # Check if base_preds and val have the same length
    if base_preds.shape[0] != len(val):
        raise ValueError("base_preds and val must have the same length.")
    # Ensure base_preds and val are numpy arrays

    
    # Check if base_preds and val have the same length
    if len(base_preds) != len(val):
        raise ValueError("base_preds and val must have the same length.")
    # Splitting features and target variable sequentially
    train_size = int(0.65 * len(val))  # Assuming the split ratio is 80-20
    X_train, y_train = base_preds[:train_size], val[:train_size]
    X_val, y_val = base_preds[train_size:], val[train_size:]

    # Define parameter grid for Random Forest
    param_grid = {
        'n_estimators': [25, 50, 100, 150, 200],  # Number of trees in the forest
        'max_depth': [None, 10, 20, 30],      # Maximum depth of the tree
        'min_samples_split': [2, 5, 8, 10, 15],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4, 6]     # Minimum number of samples required to be at a leaf node
    }

    # Initialize Random Forest regressor
    rf = RandomForestRegressor(random_state=42)

    search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=2, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)

    print("Stacking Approach")

    # Print the best estimator found
    print(search.best_estimator_)

    # Make predictions using the best model
    y_pred = search.best_estimator_.predict(X_val)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_val, y_pred)
    print("Mean Absolute Error (MAE):", mae)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error (MSE):", mse)

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Calculate Mean Absolute Percentage Error (MAPE)
    mape = mean_absolute_percentage_error(y_val, y_pred)
    print("Mean Absolute Percentage Error (MAPE):", mape)

    print("")

    return y_pred, y_val, mae, mse, rmse, mape



### <a id='toc1_4_2_'></a>[Visualization for stacking](#toc0_)

In [23]:
def plot_predictions_stack_interactive(val, y_pred, y_val):
    # Assuming val was a pandas Series or DataFrame and you want to keep its index
    if isinstance(val, pd.Series):
        val_index = val.index
    elif isinstance(val, pd.DataFrame):
        val_index = val.index
        val = val.iloc[:, 0]  # Assuming you want the first column if it's a DataFrame
    else:
        val_index = np.arange(len(val))

    # Convert lists to numpy arrays
    val = np.asarray(val).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    y_val = np.asarray(y_val).reshape(-1)

    # Plot predicted and actual values
    # Splitting features and target variable sequentially
    train_size = int(0.8 * len(val))  # Assuming the split ratio is 80-20
    pred_trace = go.Scatter(x=val_index[train_size:], y=y_pred, mode='lines', name='Predicted', line=dict(color='red'))
    val_trace = go.Scatter(x=val_index[train_size:], y=y_val, mode='lines', name='Actual', line=dict(color='blue'))

    # Create the layout
    layout = go.Layout(title="Interactive Plot",
                       xaxis=dict(title='Timestamp'),
                       yaxis=dict(title='Value'))

    # Combine traces into a list
    data = [pred_trace, val_trace]

    # Create the figure
    fig = go.Figure(data=data, layout=layout)

    # Show the interactive plot
    fig.show()

### <a id='toc1_4_3_'></a>[Running of stacking Approach](#toc0_)

In [24]:
def generate_stacked_predictions(best_models, unique_id, predicted_df_final,  original_values_df):
    base_preds = []
     # Extract directory and file name from unique_id
    dir_name, file_name = unique_id.split('/')
    print(dir_name, file_name)
    print(best_models)
    
    for model in best_models:
        print('model:', model)
        if model.lower() == "random_forest":
                # Find the respective row in predicted_df
                model_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
                # Get the predicted value for the model
                pred_value = model_row[model].iloc[0]

                print(pred_value)
                # Append the predicted value to base_preds
                base_preds.append(pred_value)
        elif model.lower() == "exponential_smoothing":
            # Find the respective row in predicted_df
                model_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
                # Get the predicted value for the model
                pred_value = model_row[model].iloc[0]

                print(pred_value)
                # Append the predicted value to base_preds
                base_preds.append(pred_value)
        elif model.lower() == "xgboost":
            # Find the respective row in predicted_df
                model_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
                # Get the predicted value for the model
                pred_value = model_row[model].iloc[0]

                print(pred_value)
                # Append the predicted value to base_preds
                base_preds.append(pred_value)
        elif model.lower() == "vae":
            # Find the respective row in predicted_df
                model_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
                # Get the predicted value for the model
                pred_value = model_row[model].iloc[0]

                print(pred_value)
                # Append the predicted value to base_preds
                base_preds.append(pred_value)

    if len(best_models) == 1:
        # If only one model is selected, return its prediction directly
        return base_preds[0]

    else:
        val_row = original_values_df[(original_values_df['dir'] == dir_name) & (original_values_df['file_name'] == file_name)]
        #print(val_row)
        val = val_row['values'].iloc[0]
        #print(val)
        base_preds = np.stack(base_preds, axis=-1)
        y_pred, y_val, mae, mse, rmse, mape = stacked_model_predictions(val, base_preds)
        plot_predictions_stack_interactive(val, y_pred, y_val)
        return y_pred

In [25]:
column_mapping = {
    'series_length':'series_length',
    'stability': 'stability',
    'lumpiness': 'lumpiness',
    'crossing.points.fraction': 'crossing_points',  
    'flat.spots.fraction': 'flat_spots',  
    'nonlinearity': 'nonlinearity',
    'ur.kpss': 'unitroot_kpss',
    'ur.pp': 'unitroot_pp',
    'arch.lm': 'arch_lm',
    'ACF1': 'x_acf1',
    'ACF10.SS': 'x_acf10',
    'ACF.seas': 'seas_acf1',
    'PACF10.SS': 'x_pacf5',
    'PACF.seas': 'seas_pacf',
    'hurst': 'hurst'
    #'ensemble_size':'ensemble_size'
}

In [26]:
# Filter the DataFrame
df = df[[col for col in column_mapping.values() if col in df.columns]]
df

,series_length,stability,lumpiness,crossing_points,flat_spots,nonlinearity,unitroot_kpss,unitroot_pp,arch_lm,x_acf1,x_acf10,seas_acf1,x_pacf5,seas_pacf,hurst
0,1647,0.011183,1.921403,315,488,0.278744,0.181736,-1839.697103,0.005727,0.030553,0.026503,0.000060,0.024306,-0.020632,0.390779
1,4032,0.076766,0.355272,1691,505,0.067505,0.489215,-654.999557,0.587577,0.818554,1.326405,0.116130,0.733038,0.028988,0.950880
2,4037,0.834827,0.345937,1638,456,1.939950,12.161875,-8.202838,0.996375,0.988133,9.567909,0.320249,1.223311,0.014974,1.089914
3,4032,0.029572,0.348292,769,777,0.217334,0.200876,-992.787092,0.366378,0.726203,0.896081,0.084554,0.659281,0.011612,0.802616
4,7888,0.698239,0.043476,501,175,0.004126,15.832975,-157.619180,0.918206,0.977310,8.132260,0.523375,1.052754,-0.014149,1.085836


## <a id='toc1_6_'></a>[Ensemble size finding model running](#toc0_)

In [27]:
import os
import pickle

# Directory containing the pickle files
model_directory = "model_pickle/ensemble_size"

# List all files in the directory
pickle_files = [file for file in os.listdir(model_directory) if file.endswith('.pkl')]

if len(pickle_files) == 0:
    print("No pickle files found in the directory.")
else:
    # Load the first pickle file found
    model_filename = os.path.join(model_directory, pickle_files[0])
    with open(model_filename, 'rb') as f:
        model = pickle.load(f)

    # Make predictions on your data
    predictions = model.predict(df)  # Replace df_data with your actual DataFrame containing the data

    # The variable 'predictions' now contains the predicted y values for your data
    print(predictions)

[1 1 1 1 1]


In [28]:
best_models=['vae','exponential_smoothing','xgboost']

In [29]:
unique_id='realAdExchange/exchange-4_cpm_results.csv'

## <a id='toc1_7_'></a>[ Running the final function and producing results](#toc0_)

In [30]:
y_pred=generate_stacked_predictions(best_models, unique_id, predicted_df_final, original_values_df)

realAdExchange exchange-4_cpm_results.csv
['vae', 'exponential_smoothing', 'xgboost']
model: vae
[0.428944811047, 0.469532759242, 0.455640228997, 0.407852218304, 0.304634826503, 0.326486209365, 0.378021864107, 0.429690650383, 0.421697349432, 0.484168498724, 0.444613236765, 0.43423724996, 0.390200679323, 0.397312948807, 0.366870209429, 0.417927008071, 0.340769890366, 0.26066963881, 0.299583640201, 0.289433049501, 0.290487908915, 0.270638177333, 0.294909414909, 0.23479249864399998, 0.22450159758599997, 0.20823722558400004, 0.230379338664, 0.21707267603199998, 0.270446384561, 0.784133429713, 0.225890586909, 0.624101601009, 0.540090192297, 0.636013037099, 0.538367473326, 0.526973008519, 0.423723562203, 0.439474601966, 0.436875837086, 0.599085251556, 0.598101988004, 0.574486414844, 0.654409647061, 0.676325505834, 0.625670543758, 0.518763257894, 0.306509016977, 0.22254324420600002, 0.238742197519, 0.216867313949, 0.260681321168, 0.34920246888, 0.485340489563, 0.500499372968, 0.499002522919, 

Stacking Approach
RandomForestRegressor(max_depth=30, min_samples_leaf=6, min_samples_split=15,
                      n_estimators=25, random_state=42)
Mean Absolute Error (MAE): 0.43647309815916524
Mean Squared Error (MSE): 0.5420535383518801
Root Mean Squared Error (RMSE): 0.7362428528358561
Mean Absolute Percentage Error (MAPE): 1.0557619273711232

